In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import time

In [2]:
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [3]:
pc = Pinecone()
index_name = 'codedoc'

In [4]:
all_documents = []

law_folders = [folder for folder in Path("../../내파일/").iterdir() 
                   if folder.is_dir() and folder.name.endswith('법령')]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
                
for folder in law_folders:
    docx_files = list(folder.glob('*.docx'))
    for docx_path in docx_files:
        loader = Docx2txtLoader(str(docx_path))
        document_list = loader.load_and_split(text_splitter=text_splitter)
        
        all_documents.extend(document_list)


In [8]:
if all_documents:
    # API 요청 제한을 피하기 위해 배치 크기를 더 보수적으로 줄입니다.
    batch_size = 30 
    # 최대 재시도 횟수
    max_retries = 5 
    
    total_batches = (len(all_documents) + batch_size - 1) // batch_size
    print(f"\n총 {len(all_documents)}개의 텍스트 조각을 {total_batches}개의 배치로 나누어 업로드합니다...")

    for i in range(0, len(all_documents), batch_size):
        # 처리할 현재 배치를 선택합니다.
        batch = all_documents[i:i + batch_size]
        
        for attempt in range(max_retries):
            try:
                # 현재 배치를 Pinecone에 업로드합니다.
                PineconeVectorStore.from_documents(
                    documents=batch,
                    embedding=embedding,
                    index_name=index_name
                )
                # 성공 시 재시도 루프를 빠져나갑니다.
                break 
            except Exception as e:
                if "429" in str(e): # API Rate Limit 에러인 경우
                    # 2, 4, 8, 16, 32초 순으로 대기 시간이 늘어납니다.
                    wait_time = 2 ** (attempt + 1) 
                    print(f"배치 {i//batch_size + 1} 처리 중 API 제한 발생. {wait_time}초 후 재시도합니다... (시도 {attempt + 1}/{max_retries})")
                    time.sleep(wait_time)
                else: # 다른 종류의 에러인 경우
                    print(f"배치 {i//batch_size + 1} 처리 중 예상치 못한 오류 발생: {e}")
                    break # 이 배치는 포기하고 다음 배치로 넘어갑니다.
            
            if attempt == max_retries - 1:
                print(f"배치 {i//batch_size + 1} 업로드 최종 실패. 다음 배치를 계속 진행합니다...")
        
        # 각 배치가 성공적으로 끝나도, 다음 배치를 위해 잠시 쉬어줍니다.
        time.sleep(1.5)

    print("모든 배치 업로드가 성공적으로 완료되었습니다!")
else:
    print("처리할 문서가 없습니다.")


총 7755개의 텍스트 조각을 259개의 배치로 나누어 업로드합니다...
모든 배치 업로드가 성공적으로 완료되었습니다!
